In [0]:
%sh
rm -r /dbfs/hyperopt_lab
mkdir /dbfs/hyperopt_lab
wget -O /dbfs/hyperopt_lab/penguins.csv https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/penguins.csv

rm: cannot remove '/dbfs/hyperopt_lab': No such file or directory
--2025-01-23 14:33:01--  https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/penguins.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9533 (9.3K) [text/plain]
Saving to: ‘/dbfs/hyperopt_lab/penguins.csv’

     0K .........                                             100%  523K=0.02s

2025-01-23 14:33:01 (523 KB/s) - ‘/dbfs/hyperopt_lab/penguins.csv’ saved [9533/9533]



In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
   
data = spark.read.format("csv").option("header", "true").load("/hyperopt_lab/penguins.csv")
data = data.dropna().select(col("Island").astype("string"),
                          col("CulmenLength").astype("float"),
                          col("CulmenDepth").astype("float"),
                          col("FlipperLength").astype("float"),
                          col("BodyMass").astype("float"),
                          col("Species").astype("int")
                          )
display(data.sample(0.2))
   
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
print ("Training Rows:", train.count(), " Testing Rows:", test.count())

Island,CulmenLength,CulmenDepth,FlipperLength,BodyMass,Species
Torgersen,34.1,18.1,193.0,3475.0,0
Torgersen,42.0,20.2,190.0,4250.0,0
Torgersen,37.8,17.3,180.0,3700.0,0
Torgersen,34.6,21.1,198.0,4400.0,0
Torgersen,42.5,20.7,197.0,4500.0,0
Dream,37.2,18.1,178.0,3900.0,0
Dream,39.2,21.1,196.0,4150.0,0
Dream,42.2,18.5,180.0,3550.0,0
Dream,39.8,19.1,184.0,4650.0,0
Dream,36.5,18.0,182.0,3150.0,0


Training Rows: 238  Testing Rows: 104


* create the objective function

  uses the penguin data to rain a classification model that predicts the species of a penguin based on its location and measurements

In [0]:
from hyperopt import STATUS_OK
import mlflow
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, MinMaxScaler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
  
def objective(params):
    # Train a model using the provided hyperparameter value
    catFeature = "Island"
    numFeatures = ["CulmenLength", "CulmenDepth", "FlipperLength", "BodyMass"]
    catIndexer = StringIndexer(inputCol=catFeature, outputCol=catFeature + "Idx")
    numVector = VectorAssembler(inputCols=numFeatures, outputCol="numericFeatures")
    numScaler = MinMaxScaler(inputCol = numVector.getOutputCol(), outputCol="normalizedFeatures")
    featureVector = VectorAssembler(inputCols=["IslandIdx", "normalizedFeatures"], outputCol="Features")
    mlAlgo = DecisionTreeClassifier(labelCol="Species",    
                                    featuresCol="Features",
                                    maxDepth=params['MaxDepth'], maxBins=params['MaxBins'])
    pipeline = Pipeline(stages=[catIndexer, numVector, numScaler, featureVector, mlAlgo])
    model = pipeline.fit(train)
      
    # Evaluate the model to get the target metric
    prediction = model.transform(test)
    eval = MulticlassClassificationEvaluator(labelCol="Species", predictionCol="prediction", metricName="accuracy")
    accuracy = eval.evaluate(prediction)
      
    # Hyperopt tries to minimize the objective function, so you must return the negative accuracy.
    return {'loss': -accuracy, 'status': STATUS_OK}

* define a search space, specify the Hyperopt algorithm, use hyperopt.fmin

In [0]:
from hyperopt import fmin, tpe, hp
    
# Define a search space for two hyperparameters (maxDepth and maxBins)
search_space = {
    'MaxDepth': hp.randint('MaxDepth', 10),
    'MaxBins': hp.choice('MaxBins', [10, 20, 30])
}
  
# Specify an algorithm for the hyperparameter optimization process
algo=tpe.suggest
  
# Call the training function iteratively to find the optimal hyperparameter values
argmin = fmin(
  fn=objective,
  space=search_space,
  algo=algo,
  max_evals=6)
  
print("Best param values: ", argmin)

  0%|          | 0/6 [00:00<?, ?trial/s, best loss=?]

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

 17%|█▋        | 1/6 [00:56<04:43, 56.61s/trial, best loss: -0.4519230769230769]

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

 33%|███▎      | 2/6 [01:43<03:23, 50.80s/trial, best loss: -0.9807692307692307]

...

 67%|██████▋   | 4/6 [03:15<01:35, 47.70s/trial, best loss: -0.9807692307692307]

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

 83%|████████▎ | 5/6 [04:04<00:48, 48.09s/trial, best loss: -0.9807692307692307]

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 6/6 [04:49<00:00, 48.26s/trial, best loss: -0.9807692307692307]
Best param values:  {'MaxBins': 0, 'MaxDepth': 3}
